In [6]:
import re
import math

In [7]:
# to open and read our tweets to begin the preprocessing step
raw_text = open('tweets.csv').read()
tab_seperated = [item.split('\t') for item in raw_text.split('\n') if len(item.split('\t')) >= 4]

In [8]:
data = {}
for i in range(len(tab_seperated)):
    #extracting the filtered tweets and IDs
    data[tab_seperated[i][1]] = tab_seperated[i][4]

#Preprocessing and cleaning
data_index = data.copy()    
for tweet in data_index.keys():
    data_index[tweet] = data_index[tweet].lower()
    data_index[tweet] = re.sub('\[newline\]', ' ' , data_index[tweet])
    data_index[tweet] = re.sub('https?[^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub('[@#][^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'[0-9][^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'\w+\.[^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'[^a-zäöüß\s]', ' ', data_index[tweet])
    data_index[tweet] = re.sub(r'[^\w\s]', ' ' , data_index[tweet])
    
terms = {}
for num,tweet in data_index.items():
    for word in tweet.split():
        if word in terms:
            terms[word] += 1
        else:
            terms[word] = 1

In [9]:
idfDict = {}
N = len(data_index)
idfDict = terms.copy()
for word, val in idfDict.items():
        idfDict[word]= 1 +math.log10(N/float(val))

In [30]:
def cosine(text1, text2):
    import math
    import re
    def wd(t1, t2):
        wd = {}
        t1 = t1.split()
        t2 = t2.split()
        for i in t1:
            wd[i] = 0.0
        for i in t2:
            wd[i] = 0.0
        return t1, t2, wd
    
    def TF(wordDict, txt):
        tfDict = {}
        for word, count in wordDict.items():
            for i in txt:
                if word == i:
                    count +=1
            if count > 0:
                tfDict[word]=1+math.log10(count)
            else:
                tfDict[word] = 0.0
        return tfDict

    def TF_IDF(tfs, idfs):
        tfidf = {}
        for word, val in tfs.items():
            tfidf[word] = val*idfs[word]
        return tfidf

    def length(tf):
        return math.sqrt(sum([(i**2) for i in list(tf.values())]))
    
    def dot(v1,v2):
        new_v = []
        for i in range(len(v1)):
            new_v.append(v1[i]*v2[i])
        return sum(new_v)
    
    txt1, txt2, wd = wd(text1, text2)
    if len(txt1) == 0 or len(txt2) == 0:
        return 0
    tf1 = TF(wd, txt1)
    tf2 = TF(wd, txt2)
    tf_idf1 = TF_IDF(tf1, idfDict)
    tf_idf2 = TF_IDF(tf2, idfDict)
    l1 = length(tf_idf1)
    l2 = length(tf_idf2)
    if l1 == 0 or l2 == 0:
        return 0
#     norm_tfidf1 = [(i/l1) for i in tf_idf1.values()]
#     norm_tfidf2 = [(i/l2) for i in tf_idf2.values()]
#     return round(dot(norm_tfidf1, norm_tfidf2),4)
    return round(dot(list(tf_idf1.values()), list(tf_idf2.values()))/(l1*l2),4)

In [31]:
def top_100(tweet):
    top = []
    for id, doc in data_index.items():
        if id != tweet:
            cos = cosine(data_index[tweet], doc)
            top.append((cos,id,data[id]))
    return sorted(top)[::-1][:100]

In [24]:
data['965734992633565184']

'@knakatani @ChikonJugular @joofford @SteveBlogs11 https://t.co/WHtaRYGNSY says lifetime risk of cervical cancer in Japan is 1 in 100.  That means HPV is endemic in Japan, and screening is not working well.'

In [25]:
top = top_100('965734992633565184')
for i in top:
    print(i)

(0.3896, '982642444943605760', "@beautyloveknow1 @VABVOX Yeah, cervical cancer is so rare that the lifetime risk of getting it is 1 in 161... and cervical isn't the only kind of cancer HPV causes.[NEWLINE][NEWLINE]Please vaccinate your kids against HPV-caused cancers.  Getting the vaccine is far safer than not getting it.")
(0.3413, '988887135699648512', "@vikki_r To prevent the cancers HPV causes?[NEWLINE]Lifetime risk of cervical cancer in US is 1 in 161 without vaccination.[NEWLINE]The vaccine's about 80-90% effective against cervical and other HPV-caused cancers.  Getting the vaccine can prevent some awful problems down the road.")
(0.3389, '968982710222970880', "@RustyPee4 @williambolivar7 HPV 16 and 18 account for ~70% of cervical cancers, that's true.[NEWLINE]HPV 31, 33, 45, 52, 58 account for another ~16%.[NEWLINE]See https://t.co/ov2oS2Jdaa[NEWLINE][NEWLINE]Under about 10% of cervical cancer is not caused by HPV, depending.")
(0.3279, '968978184438915072', '@RustyPee4 @william

In [26]:
for num, key in enumerate(data_index.keys()):
    if num == 8:
        print(num, data[key], key, data_index[key])

8 @rip_tear Tumors?[NEWLINE]Dat leg is straight mccain 965481507266727936   tumors  dat leg is straight mccain


In [27]:
top = top_100('965481507266727936')
for i in top:
    print(i)

(0.412, '974741740807737344', "Leg' auf[NEWLINE]#KayasGroßeKinoshow")
(0.299, '960300385738031105', '@Lenqrad du und straight als ob')
(0.2476, '960119588309688320', "Did McCain's Brain Tumor Come From Cellphone Use? https://t.co/0u4HkeKHFZ via @mercola")
(0.2446, '997875345767444480', 'HE HAS TUMORS???')
(0.2438, '988902240260886528', 'Trump is obese, not large.[NEWLINE][NEWLINE]Get it straight. https://t.co/pj8omIV710')
(0.2432, '976800329839898625', 'Show a little more leg, honey 💕[NEWLINE].[NEWLINE].[NEWLINE].[NEWLINE]#leg #woman #sculpture #clay #ceramic #body #stoneware #love #figure #female #wip #inthestudio #newwork #exhibition #black #painting #clairedelune #art #artiststudio #keramik https://t.co/YVMiCXsvP6 https://t.co/SGdsX9elW5')
(0.2394, '962780166966726656', '@the_necrosis @KieferGabriele @ChristianNbg Was soll dat heißen? 🙊')
(0.2272, '995409237324034049', '@mounira_ber تقريبا كل ال Solid tumors. المركز في ألمانيا, باد إمستال  https://t.co/ohjpgAjZkr')
(0.2208, '9864343

In [28]:
for num, key in enumerate(data_index.keys()):
    if num == 22:
        print(num, data[key], key, data_index[key])

22 Und: Syphilis kommt wieder.[NEWLINE]Ja, eine P(R)EP/Therapie kann das HIV Übertragungs-/Infektionsrisiko auf (nahezu) null senken.[NEWLINE]Aber Syphilis, der kleine Ficker, ist schon längst wieder da.[NEWLINE]Syphilis tut am Anfang nicht weh. Aber am Ende beißt sie einem doch in den Arsch... 965711648278437888 und  syphilis kommt wieder  ja  eine p r ep therapie kann das hiv übertragungs  infektionsrisiko auf  nahezu  null senken  aber syphilis  der kleine ficker  ist schon längst wieder da  syphilis tut am anfang nicht weh  aber am ende beißt sie einem doch in den  


In [29]:
top = top_100('965711648278437888')
for i in top:
    print(i)

(0.2481, '1004722085963366400', '@the_necrosis Der beißt nicht 😉')
(0.1915, '992450802198417413', '@mastermikeg @FreeBourgeoisie @FrankBreuer12 @SteinbachErika Ich verstehe die Welt nicht mehr. Aber wer tut das schon? Aber da wo der Hass ist kann keine Liebe 💕 sein. #Ellwangen #4Mei #5Mei #love ❤️❤️❤️❤️')
(0.1911, '966642858836021249', 'Was der Olympiasieger sagt. Tut ja schon weh beim hinsehen.... #biathlon #horror https://t.co/8WnrIpODzu')
(0.1891, '982040536419840000', 'Ich habe gerade erfahren, dass heute ein Tag früher ist, als ich dachte! Es ist erst Donnerstag auf Freitag, nicht Freitag auf Samstag.[NEWLINE]Und da ich meinte, dass das letzte Kapitel Samstag kommt, es aber ja doch Fr. kommt hab ich den Tag anscheinend schon Anfang der Woche verloren')
(0.1872, '970634586085617664', '#Liebe tut weh. Also muss es auch eine #Krankheit sein.')
(0.1861, '1020076576233480192', '@el_loko74 Ist schon wieder Fußball hier? :P')
(0.1855, '997990022291828736', '@LottaFlodda @DoroRoemer @Rotl